# 神经网络，PyTorch示例

一层(隐含层)全连接神经网络

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [2]:
# Hyper Parameters，定义训练神经网络的超参数
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [4]:
# MNIST Dataset 
train_dataset = dsets.MNIST(root='./data', 
                            train=True, 
                            transform=transforms.ToTensor(),  
                            download=True)

test_dataset = dsets.MNIST(root='./data', 
                           train=False, 
                           transform=transforms.ToTensor())

# Data Loader (Input Pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [5]:
# Neural Network Model (1 hidden layer)，定义一个只有一层隐含层的简单神经网络
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out
    
net = Net(input_size, hidden_size, num_classes) #实例化上文定义的神经网络

    
# Loss and Optimizer，定义损失函数和优化方式，此处分别选择softmax函数 (即求交叉熵)，和Adam优化器 (也可选择SGD优化器)
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [12]:
# Train the Model，训练神经网络模型
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # Convert torch tensor to Variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        
        # Forward + Backward + Optimize
        # 一步优化步骤里包括：1）根据网络权值前向转播 2）根据损失求导和链式法则反向传播 3）根据梯度更新网络权值
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1170
Epoch [1/5], Step [200/600], Loss: 0.1271
Epoch [1/5], Step [300/600], Loss: 0.1998
Epoch [1/5], Step [400/600], Loss: 0.1757
Epoch [1/5], Step [500/600], Loss: 0.0851
Epoch [1/5], Step [600/600], Loss: 0.0688
Epoch [2/5], Step [100/600], Loss: 0.0305
Epoch [2/5], Step [200/600], Loss: 0.0289
Epoch [2/5], Step [300/600], Loss: 0.0467
Epoch [2/5], Step [400/600], Loss: 0.0481
Epoch [2/5], Step [500/600], Loss: 0.0854
Epoch [2/5], Step [600/600], Loss: 0.0312
Epoch [3/5], Step [100/600], Loss: 0.0300
Epoch [3/5], Step [200/600], Loss: 0.0546
Epoch [3/5], Step [300/600], Loss: 0.0479
Epoch [3/5], Step [400/600], Loss: 0.0426
Epoch [3/5], Step [500/600], Loss: 0.0296
Epoch [3/5], Step [600/600], Loss: 0.0213
Epoch [4/5], Step [100/600], Loss: 0.0135
Epoch [4/5], Step [200/600], Loss: 0.0108
Epoch [4/5], Step [300/600], Loss: 0.0352
Epoch [4/5], Step [400/600], Loss: 0.0802
Epoch [4/5], Step [500/600], Loss: 0.0235
Epoch [4/5], Step [600/600], Loss:

In [13]:
# Test the Model，测试训练好的模型
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

# Save the Model，存储训练好的模型
torch.save(net.state_dict(), 'model.pkl')

Accuracy of the network on the 10000 test images: 97 %
